# Problem Set 2 - Part 2: Inference

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the following instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

In this part you will work on a problem from the FraCaS test suite. The goal of this problem set is to extend the grammar for Cooper storage from the previous part, apply the sentences to a theorem prover and a model builder, analyse their results and conclude about the inference. Since every sentence may be translated to several formulae there will also be several inference steps and thus several results.



### Pre-requisite knowledge

From `problem-set-1`:
- First order logic
- Lambda calculus
- Feature unification context free grammar

From `problem-set-2-part1.ipynb`:
- Cooper storage



In [1]:
# This task requires NLTK and Jupyter Notebook (IPython package).
import nltk
from nltk.grammar import FeatureGrammar
from nltk.sem import cooper_storage as cs

from utils import display_latex, display_translation, display_tree, display, Markdown
from utils2 import sem_parser, evaluate_sentences, syntax, syntax_notv

read_expr = nltk.sem.Expression.fromstring


### 1.1 Extending the grammar [6 marks]

Extend the grammar below so that it covers the sentences of the inference problem based on FraCaS problem 057.

Several solutions are possible, also some that would not result in entailment. Our lexical knowledge about words leads us how to formalise them in first order logic. Therefore, make sure that you handle the PP attachment in a away that you will be able to show entailments between examples.

In [39]:
sentences = [
    'a Portuguese delegate published the result on climate',
    'a Portuguese delegate published a result',
    'a Portuguese delegate published the result',
    'a Portuguese published a result',
]

In [120]:
fcfg_storage_base = r"""
% start S

S[SEM=[CORE=<?subj(?vp)>, STORE=(?b1+?b2)]] -> NP[NUM=?n,SEM=[CORE=?subj, STORE=?b1]] VP[NUM=?n,SEM=[CORE=?vp, STORE=?b2]]

Nom[NUM=?n,SEM=?s] -> N[NUM=?n,SEM=?s]

VP[NUM=?n,SEM=?s] -> IV[NUM=?n,SEM=?s]
VP[NUM=?n,SEM=[CORE=<?v(?obj)>, STORE=(?b1+?b2)]] -> TV[NUM=?n,SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]]
VP[NUM=?n,SEM=[CORE=<?v(?pp)(?obj)>, STORE=(?b1+?b2+?b3)]] -> DTV[NUM=?n,SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]] PP[+TO,SEM=[CORE=?pp, STORE=?b3]]

PP[+TO, SEM=[CORE=?np, STORE=?b1]] -> P[+TO] NP[SEM=[CORE=?np, STORE=?b1]]
"""

fcfg_storage_lexicon = r"""
PropN[NUM=sg,SEM=[CORE=<\P.P(angus)>, STORE=(/)]] -> 'Angus'
PropN[NUM=sg,SEM=[CORE=<\P.P(cyril)>, STORE=(/)]] -> 'Cyril'
PropN[NUM=sg,SEM=[CORE=<\P.P(irene)>, STORE=(/)]] -> 'Irene'

Det[NUM=sg,SEM=[CORE=<\P Q.all x.(P(x) -> Q(x))>, STORE=(/)]] -> 'every'
Det[NUM=sg,SEM=[CORE=<\P Q.exists x.(P(x) & Q(x))>, STORE=(/)]] -> 'a'

N[NUM=sg,SEM=[CORE=<\x.library(x)>, STORE=(/)]] -> 'library'
N[NUM=sg,SEM=[CORE=<\x.girl(x)>, STORE=(/)]] -> 'girl'
N[NUM=sg,SEM=[CORE=<\x.boy(x)>, STORE=(/)]] -> 'boy'
N[NUM=sg,SEM=[CORE=<\x.book(x)>, STORE=(/)]] -> 'book'

IV[NUM=sg,SEM=[CORE=<\x.smile(x)>, STORE=(/)],TNS=pres] -> 'smiles' 

TV[NUM=sg,SEM=[CORE=<\X x.X(\y.read(x,y))>, STORE=(/)],TNS=pres] -> 'reads'

DTV[NUM=sg,SEM=[CORE=<\Y X x.X(\z.Y(\y.give(x,y,z)))>, STORE=(/)],TNS=pres] -> 'gives'

P[+to] -> 'to'
""" 

fcfg_storage_np = r"""
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?np, @x)>+?b1)]] -> PropN[NUM=?n,SEM=[CORE=?np, STORE=?b1]]
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?det(?nom), @x)>+?b1+?b2)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b2]]
"""

The following code resolves the readings from the Cooper storage. We collect the readings in a dictionary where the key is a sentence string and the value is a list of readings.

In [154]:
# your answers
fcfg_storage_answers_1 = r"""
### Replace X with their proper representations
ADJ[NUM=sg,SEM=[CORE=<\P x.(P(x) & portuguese(x))>, STORE=(/)]] -> 'Portuguese'

N[NUM=sg,SEM=[CORE=<\x.portuguese(x)>, STORE=(/)]] -> 'Portuguese'
N[NUM=sg,SEM=[CORE=<\x.delegate(x)>, STORE=(/)]] -> 'delegate'
N[NUM=sg,SEM=[CORE=<\x.result(x)>, STORE=(/)]] -> 'result'
N[NUM=sg,SEM=[CORE=<\x.climate(x)>, STORE=(/)]] -> 'climate'

TV[NUM=sg,SEM=[CORE=<\X x.X(\y.publish(x,y))>, STORE=(/)],TNS=past] -> 'published'

Det[SEM=[CORE=<\P Q.exists x.(P(x) & all y.(P(y) -> x=y) & Q(x))>, STORE=(/)]] -> 'the'

P[+on] -> 'on' 
#P[+LOC,SEM=[CORE=<\X x.X(\y.on(x,y))>, STORE=(/)]] -> 'on' 
P[+LOC,SEM=[CORE=<\P y exists x.(on(x,y) & P(x))>, STORE=(/)]] -> 'on' #Mehdis
PP[+ON, SEM=[CORE=?n, STORE=?b1]] -> P[+ON] N[SEM=[CORE=?n, STORE=?b1]]
#PP[+ON, SEM=[CORE=<?p(?nom)>, STORE=?b1+?b2]] -> P[SEM=[CORE=?p, STORE=?b1]] Nom[SEM=[CORE=?nom, STORE=?b2]] #Mehdis förslag

NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?np, @x)>+?b1)]] -> N[NUM=?n,SEM=[CORE=?n, STORE=?b1]]
VP[NUM=?n,SEM=[CORE=<?v(?obj)(?pp)>, STORE=(?b1+?b2+?b3)]] -> TV[NUM=?n,SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]] PP[+ON,SEM=[CORE=?pp, STORE=?b3]]
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?det(?adj(?nom)), @x)>+?b1+?b2+?b3)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] ADJ[NUM=?n,SEM=[CORE=?adj, STORE=?b2]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b3]]
## Extend the other missig rules too
"""

# this is going to add new rules to the syntax:
fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon + fcfg_storage_answers_1
new_syntax = FeatureGrammar.fromstring(fcfg_storage)

ValueError: Unable to parse line 51: P[+LOC,SEM=[CORE=<\P y exists x.(on(x,y) & P(x))>, STORE=(/)]] -> 'on' #Mehdis
Error parsing feature structure
    P[+LOC,SEM=[CORE=<\P y exists x.(on(x,y) & P(x))>, STORE=(/)]] -> 'on' #Mehdis
                                                                           ^ Expected open bracket or identifier

In [152]:
sentence_readings = sem_parser(sentences, new_syntax, verbose=False, is_cs=True)

# print all readings
for i, (sent, semreps) in enumerate(sentence_readings.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

1. a Portuguese delegate published the result on climate


"1": $\exists\ x.(result(x)\ \land\ \forall\ y.(result(y)\ \to\ (x\ =\ y))\ \land\ \exists\ z_{336}.(delegate(z_{336})\ \land\ portuguese(z_{336})\ \land\ publish(\lambda\ z_{335}.climate(z_{335}),x,z_{336})))$

"2": $\exists\ x.(delegate(x)\ \land\ portuguese(x)\ \land\ \exists\ z_{338}.(result(z_{338})\ \land\ \forall\ y.(result(y)\ \to\ (z_{338}\ =\ y))\ \land\ publish(\lambda\ z_{337}.climate(z_{337}),z_{338},x)))$

"3": $\exists\ x.(result(x)\ \land\ \forall\ y.(result(y)\ \to\ (x\ =\ y))\ \land\ \exists\ z_{340}.(delegate(z_{340})\ \land\ portuguese(z_{340})\ \land\ publish(\lambda\ z_{339}.climate(z_{339}),x,z_{340})))$

"4": $\exists\ x.(delegate(x)\ \land\ portuguese(x)\ \land\ \exists\ z_{342}.(result(z_{342})\ \land\ \forall\ y.(result(y)\ \to\ (z_{342}\ =\ y))\ \land\ publish(\lambda\ z_{341}.climate(z_{341}),z_{342},x)))$

...

"1": $\exists\ x.(result(x)\ \land\ \forall\ y.(result(y)\ \to\ (x\ =\ y))\ \land\ \exists\ z_{345}.(delegate(z_{345})\ \land\ portuguese(z_{345})\ \land\ publish(z_{345},x)))$

"2": $\exists\ x.(delegate(x)\ \land\ portuguese(x)\ \land\ \exists\ z_{346}.(result(z_{346})\ \land\ \forall\ y.(result(y)\ \to\ (z_{346}\ =\ y))\ \land\ publish(x,z_{346})))$

4. a Portuguese published a result


"1": $\exists\ x.(result(x)\ \land\ \exists\ z_{347}.(portuguese(z_{347})\ \land\ publish(z_{347},x)))$

"2": $\exists\ x.(portuguese(x)\ \land\ \exists\ z_{348}.(result(z_{348})\ \land\ publish(x,z_{348})))$

In [90]:
# make sure understanding this data structure:
sentence_readings

{'a Portuguese delegate published the result on climate': [],
 'a Portuguese delegate published a result': [<ExistsExpression exists x.(result(x) & exists z132.(delegate(z132) & portuguese(z132) & publish(z132,x)))>,
  <ExistsExpression exists x.(delegate(x) & portuguese(x) & exists z133.(result(z133) & publish(x,z133)))>],
 'a Portuguese delegate published the result': [<AndExpression (exists x.(result(x) & (all y.result(y) -> (x = y2))) & exists x.(delegate(x) & portuguese(x) & publish(x,x2)))>,
  <ExistsExpression exists x.(delegate(x) & portuguese(x) & exists z134.(result(z134) & (all y.result(y) -> (z134 = y2))) & publish(x,x2))>],
 'a Portuguese published a result': [<ExistsExpression exists x.(result(x) & exists z135.(portuguese(z135) & publish(z135,x)))>,
  <ExistsExpression exists x.(portuguese(x) & exists z136.(result(z136) & publish(x,z136)))>]}

### 1.2 Using Prover9 [4 marks in total]

Show which readings of (1) entail a reading of (2). **[1 mark]**

In [146]:
# There is something missing in this code. You will have to fix it, before you can run it.

prover = nltk.Prover9()

def apply_theorem_prover(premises, goals):
    for premise in premises:
        for goal in goals:
            print("Premise      : %s" % (premise))
            print("Goal         : %s" % (goal))
            print("Prover result: %s" % (...))
            print(10*'----')
            
apply_theorem_prover(...,...)

TypeError: 'ellipsis' object is not iterable

Do all the readings of (1) entail all the readings of (2)? If not, why not? **[1 mark]**

Which readings of (1) entail a reading of (4). **[1 mark]** 

In [ ]:
# There is something missing in this code. You will have to fix it, before you can run it.

apply_theorem_prover(...,...)

Do all the readings of (1) entail all the readings of (4)? If not, why not? **[1 mark]**


### 1.3 Using Mace [2 marks in total]

Show whether (1) entails (3). **[1 mark]**

In [ ]:
# There is something missing in this code. 

def apply_model_builder(premises,goals):
    for premise in premises:
        for goal in goals:
            print("Premise : %s" % (premise))
            print("Goal    : %s" % (goal))
            mc = nltk.MaceCommand(..., assumptions=...)
            ...
            print("a model:")
            print(mc.valuation)
            print(10*'...')

apply_model_builder(...,...)

Explain why. **[1 mark]**

## Marks

This part of the assignment has a total of 12 marks.